# Connectivity Analysis Pipeline

This notebook is a first shot at making a connectivity analysis pipeline using **EBRAINS** atlas services and **nilearn**. 

The pipeline will ideally contains the following steps:

- **Step 1:** <a href='#Step1'>Load fmri data from EBRAINS</a>
- **Step 2:** <a href='#Step2'>Load a parcellation from EBRAINS human brain atlas using the `siibra` client</a>
- **Step 3:** <a href='#Step3'>Use nilearn to extract signals</a>
- **Step 4:** <a href='#Step4'>Use nilearn to compute some connectivity from these signals</a>
- **Step 5:** <a href='#Step5'>Use nilearn to visualize this connectivity (as a matrix, as a graph...)</a>
- **Step 6:** <a href='#Step6'>Upload the results back to EBRAINS</a>
- **Step 7:** Visualize them using the visualization tools of EBRAINS

## Step 1: Load fmri data

Ideally this will be loaded from **EBRAINS**. 

**TODOS:**

- [ ] find and upload good datasets
- [ ] find a way to fetch them easily 

For now, we rely on **Nilearn** for this.

In [ ]:
# Do not display warnings to prettify the notebook...
import warnings
warnings.simplefilter("ignore")

In [ ]:
# import nilearn newest version (make sure it is 0.7.1 or more)
import nilearn
nilearn.__version__

We load 10 development fmri data for 10 subjects:

In [ ]:
from nilearn.datasets import fetch_development_fmri

# Ten subjects of brain development fmri data
data = fetch_development_fmri(n_subjects=10)

## Step 2: Load an atlas from EBRAINS

We rely on the `siibra` library to work with EBRAINS human brain atlas and access the *Julich-Brain Probabilistic Cytoarchitectonic Maps (v2.5)*.

**TODOS:**

- [x] Simplify the API such that the parcellation selection in right space is smoother--
- [x] Simplify the queries on regions (ex: get the mapping between region names and label values)

<p style="color: red;"><b>Warning:</b> If this is the first time you use siibra, you will have to provide an authentication token (see <a href="https://siibra-python.readthedocs.io/en/latest/authentication.html">the siibra documentation on authenticating to EBRAINS</a>).</p>

In [ ]:
!pip install siibra==0.0.8.9.dev2 

### 2.1 Fetch a parcellation object via siibra

In [ ]:
import siibra
print(siibra.__version__)
siibra.logger.setLevel("INFO") # we want to see some messages
julichbrain = siibra.parcellations.JULICH_BRAIN_PROBABILISTIC_CYTOARCHITECTONIC_MAPS_V2_5
julichbrain.description

### 2.2 Retrieve the parcellation map in MNI152 space

We load the map for this parcellation in MNI152 space. The map is a 4D image with integer values representing the labels of the parcellation. The fourth dimension splits the left and right hemisphere in this atlas. 

In [ ]:
julichbrain_map = julichbrain.get_maps(
    siibra.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC)
print(julichbrain_map.shape)

### 2.3 Understand handling of region names and labels in siibra

The labels in the map correspond to the label list used in the parcellation object, except that the latter does not include the background label, 0:

In [ ]:
import numpy as np
labels_seen = set(np.unique(julichbrain_map.dataobj))
print("Labels in the label volume which are not defined by the parcellation object:")
print(labels_seen - julichbrain.labels)

Interpreting index labels and region names in the parcellation map is fairly simple: Label indices and names can be decoded into region objects by the atlas. In fact, the underlying regiontree of the parcellation can be directly indexed with label indices.

In [ ]:
res = julichbrain.regiontree.find(30)
print(res)

In [ ]:
print("Decoding the name 'Fp1'")
region = julichbrain.get_region("Fp1")
print(repr(region))
print("This region has label {}.".format(region.labelindex))

print("Decoding the label index 107:")
print(repr(julichbrain.get_region(region.labelindex)))

print("or, alternatively, indexing the region tree with index 107:")
print(repr(julichbrain.regiontree[region.labelindex]))

To keep track during the analysis below, we construct lists of region names and indices.

In [ ]:
julichbrain.find_region(30)

In [ ]:
labels = list(julichbrain.labels)
names = [julichbrain.get_region(l).name for l in labels]

## Step 3: Use Nilearn to extract signals from parcellation and functional data

In this section we use the nilearn `NiftiLabelsMasker` to extract the signals from the functional dataset and parcellation.

In [ ]:
from IPython.display import Image
Image(filename='masker.png') 

*copyright - Image taken from the nilearn documentation.*

More information on maskers can be found in the <a href="https://nilearn.github.io/manipulating_images/masker_objects.html">nilearn online documentation</a>.

In [ ]:
from nilearn.input_data import NiftiLabelsMasker

# Use NiftiLabelsMasker to extract signals from regions
julichbrain_left = julichbrain_map.slicer[:,:,:,0]
masker = NiftiLabelsMasker(labels_img=julichbrain_left, 
                           standardize=True) # Standardize the signals
time_series = []
for func, confounds in zip(data.func, data.confounds):
    time_series.append(masker.fit_transform(func, 
                                            confounds=confounds))
time_series = np.array(time_series)
time_series.shape

We have **124** standardized time series of length **168** per subject (**10** subjects were loaded). 

We can plot them if needed:

In [ ]:
import matplotlib.pyplot as plt

subject_id = 0
fig = plt.figure(figsize=(12,4))
for i in [0,1,2]:
    plt.plot(time_series[subject_id, :, i], 
             label=names[i])
plt.legend()
plt.xlim((0, 168))
plt.xlabel("Time", fontsize=15)
plt.title(f"Signals for subject {subject_id} for three regions", fontsize=15)
plt.tight_layout()

## Step 4: Use Nilearn to compute a connectivity matrix

Here we compute the correlation between these time series:

In [ ]:
from nilearn.connectome import ConnectivityMeasure
correlation_measure = ConnectivityMeasure(kind='correlation')
correlation_matrix = correlation_measure.fit_transform(time_series)
assert correlation_matrix.shape == (10, 124, 124)

In order to visualize this matrix, we take the mean accross subject:

In [ ]:
mean_correlation_matrix = correlation_measure.mean_
assert mean_correlation_matrix.shape == (124, 124)

## Step 5: Use nilearn to visualize the connectivity

We can use **Nilearn** to visualize the connectivity, either as a matrix or as a graph:

### As a matrix

We can plot the matrix with the region names:

In [ ]:
mean_correlation_matrix.shape, len(names)

In [ ]:
from nilearn.plotting import plot_matrix
# Mask the main diagonal for visualization:
np.fill_diagonal(mean_correlation_matrix, 0)
# matrices are ordered for block-like representation
plot_matrix(mean_correlation_matrix, 
            figure=(16, 16), 
            labels=names, 
            reorder=True)

### As a graph

In [ ]:
from nilearn.plotting import plot_connectome, find_parcellation_cut_coords

# grab center coordinates for atlas labels
coordinates = find_parcellation_cut_coords(labels_img=julichbrain_map.slicer[:,:,:,0])
# plot connectome with 95% edge strength in the connectivity
plot_connectome(mean_correlation_matrix, 
                coordinates,
                edge_threshold="95%")

## Step 6: Upload the results back to ebrains

**TODOS:**

- Decide on a representation of the connectivity results
- Find how these results could be uploaded